In [2]:
from collections import Counter
import random

def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word,K):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

def document_init(documents):
    random.seed(0)
    K=3
    document_topics = [[random.randrange(K) for word in document]
                        for document in documents]
    document_topic_counts = [Counter() for _ in documents]
    topic_word_counts = [Counter() for _ in range(K)]
    topic_counts = [0 for _ in range(K)]
    document_lengths = [len(document) for document in documents]
    distinct_words = set(word for document in documents for word in document)
    V = len(distinct_words)
    D = len(documents)
    return document_topics, document_topic_counts, topic_word_counts, topic_counts,document_lengths, distinct_words, V, D, K

def fun_LDA(documents):
    for d in range(D):
        for word, topic in zip(documents[d], document_topics[d]):
            document_topic_counts[d][topic] += 1
            topic_word_counts[topic][word] += 1
            topic_counts[topic] += 1

    for iter in range(1000):
        for d in range(D):
            for i, (word, topic) in enumerate(zip(documents[d],
                                                  document_topics[d])):
                document_topic_counts[d][topic] -= 1
                topic_word_counts[topic][word] -= 1
                topic_counts[topic] -= 1
                document_lengths[d] -= 1
                new_topic = choose_new_topic(d, word,K)
                document_topics[d][i] = new_topic
                document_topic_counts[d][new_topic] += 1
                topic_word_counts[new_topic][word] += 1
                topic_counts[new_topic] += 1
                document_lengths[d] += 1
    return document_topics,document_topic_counts,topic_word_counts,topic_counts


In [2]:
import pandas as pd
origin = pd.read_excel('data.xlsx')

origin = origin.iloc[:,2]

import re
#전처리
def def_pre(origin):
    dialogue_list = []  
    for sentece in origin:
        sentece = re.sub(r'\([^)]*\)', '', str(sentece))
        sentece = re.sub('[.,?/*!"]', '', str(sentece))
        if sentece[:1] =='아'or sentece[:1] =='검':
            dialogue_list.append(sentece[2:])
    return dialogue_list
         
def def_remove_space(dialogue_list):
    strip_senten = []
    for i in dialogue_list:
        strip_senten.append(i.strip())
    return strip_senten
    
def def_remove_space2(strip_senten):
    full_senten = []  
    for i in range(len(strip_senten)):
        if( len(strip_senten[i]) > 0):
            full_senten.append(strip_senten[i])
        else:
            continue
    return full_senten

from konlpy.tag import Kkma
kkma = Kkma()
def def_kkmaNoun(full_senten):
    origin_noun_kkma = []
    for i in range(len(full_senten)):
        nouns = kkma.pos(full_senten[i])
        # print(nouns)
        etc = []
        for word in nouns:
            if (len(word[0])>1) and (word[1][0] == 'N'):
                etc.append(word[0])
            else:
                continue
        origin_noun_kkma.append(etc)
    return origin_noun_kkma

dialogue_list = def_pre(origin)
strip_senten = def_remove_space(dialogue_list)
full_senten = def_remove_space2(strip_senten)
print(full_senten)
def bind_document(documents):
    new_documents=[]
    for i in range(len(documents)-2):
        new_documents.append(documents[i]+ ' ' + documents[i+1]+ ' ' + documents[i+2])
    return new_documents

sentence_3 = bind_document(full_senten)
print("발화의 수 : ",len(sentence_3))

#명사 추출
origin_noun_kkma = def_kkmaNoun(sentence_3)
origin_noun_kkma2 = def_remove_space2(origin_noun_kkma)
dictionary_noun = []

for i in range(len(origin_noun_kkma2)):
    for j in range(len(origin_noun_kkma2[i])):
        if origin_noun_kkma2[i][j] not in dictionary_noun:
            dictionary_noun.append(origin_noun_kkma2[i][j])
            
# display(origin_noun_kkma2)
            
from gensim import corpora
import gensim

id2word = corpora.Dictionary(origin_noun_kkma2)
texts = origin_noun_kkma2
corpus = [id2word.doc2bow(text) for text in texts]
print("id2",id2word)
print("texts",texts)
print("corpus",corpus)


lda_model = []

for i in range(3,30):
    lda_model.append(gensim.models.ldamodel.LdaModel(corpus=corpus,
    id2word=id2word,
    num_topics=i,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=50,
    alpha='auto',
    per_word_topics=True))

from gensim.models import CoherenceModel
import numpy as np

Coherence_scores = []
Perplexity_scores = []

for i, lda_model_num in enumerate(lda_model):
    coherence_model_lda = CoherenceModel(model=lda_model_num, texts=origin_noun_kkma2, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print("topic 수 : ", i+3, "  Coherence Score : ", coherence_lda, "  Perplexity : ", np.exp2(-lda_model_num.log_perplexity(corpus)))
    Coherence_scores.append(coherence_lda)
    Perplexity_scores.append(np.exp2(-lda_model_num.log_perplexity(corpus))) 

print("Coherence 최대치 : ",Coherence_scores.index(max(Coherence_scores))+3)
#standardization Coherence Score
import numpy as np

co_mean = np.mean(Coherence_scores)
co_std = np.std(Coherence_scores)
per_mean = np.mean(Perplexity_scores)
per_std = np.std(Perplexity_scores)

stan = []
for i in range(len(Coherence_scores)):
    co_standard = (Coherence_scores[i] - co_mean) / co_std
    per_standard = (Perplexity_scores[i] - per_mean) / per_std
#     print("토픽 ", i+3, "개   Coherence : ", co_standard, "  Perplexity : ", per_standard)
    stan.append(co_standard - per_standard)

print("최적의 topic 수 : ", stan.index(max(stan))+3)

['이거 이 촛불 끄는 사진은 일곱 살 때 아빠가 치즈케이크를 사 왔는데 제가 자르다가 엎어졌어요', '자르다가 엎어졌어', '네', '이렇게 이렇게 자르다가 툭 해가지고 툭 날라갔어요', '툭 날라갔구나', '그래서 반쪽밖에 못 먹었어요', '반쪽밖에 못 먹었어', '네', '바닥에 떨어져서 뭉개졌어요', '아 뭉개졌구나', '치즈케이크가 축 하고', '아 축 하고 뭉개졌구나', '그리고 한복 입고 윷놀이하는 거 있잖아요', '그거는 저희는 저는 할머니 할아버지가 할머니 할아버지 외할머니 외할아버지가 다 돌아가셨어요 제가 태어나기 전에', '아 태어나기 전에 다 돌아가셨어', '그래서 사진도 없어요', '아 사진도 없구나', '네', '사진도 없어서 얼굴을 본 적이 없어요 사진으로조차', '그런데 고모할머니가 할머니라고 부르는 유일한 그러신 분인데 고모할머니 집으로 처음으로 놀러갔을 때가 다섯 살이었어요', '다섯 살이었어', '다섯 살 때 설날이었는데 그때 떡국을 먹잖아요 설날에는', '떡국을 먹는데 옆에서 사촌언니들이 윷놀이를 하고 있는 거에요', '그래서 옆으로 가서 뭐야뭐야 하면서 봤는데  떡국이 다 불어서 먹지를 못했어요', '떡국이 다 불어서 먹지 못했어', '네', '그렇구나', '그 다음에 오토바이 같은 거 타고 이 사진 있잖아요', '이 사진은 해변가에 놀러갔었는데  그 때 아빠가 이런 거 빌려가지고 엄마랑 아빠랑 저랑 셋이 타고서 한바퀴 빙 돌았었어요', '빙 돌았었어', '그리고 가족사진 여기 있잖아요', '저희 집에는 가족사진이 없어요', '가족사진이 없어', '네', '찍으러 가지를 못해서요', '찍으러 가지는 못했구나', '사진관에 가잖아요', '근데 사진관에 가려면 힘들어요', '아 사진관에 가려면 힘들어', '왜냐면 엄마가  엄마아빠 두 분 다 장애가 있으신데 사진관이 이 근처에는 없잖아요', '그래서 차를 타고가거나 해야 하는데 차도 없고 하니까 가기가 힘들어요', '아 가기가 힘들구나', '가족사진이 없어요', '가족사진이 없

topic 수 :  3   Coherence Score :  0.5246419235814243   Perplexity :  28.96405809015244
topic 수 :  4   Coherence Score :  0.5131802713706101   Perplexity :  28.96839376630744
topic 수 :  5   Coherence Score :  0.5094412657891229   Perplexity :  27.51500281967652
topic 수 :  6   Coherence Score :  0.4713716488810711   Perplexity :  28.318651955832532
topic 수 :  7   Coherence Score :  0.4756253162560878   Perplexity :  29.01765488535056
topic 수 :  8   Coherence Score :  0.42574868003305644   Perplexity :  27.83527371626677
topic 수 :  9   Coherence Score :  0.4631381428437412   Perplexity :  27.262233112783417
topic 수 :  10   Coherence Score :  0.4577559469976219   Perplexity :  27.778745647758498
topic 수 :  11   Coherence Score :  0.41412517419196626   Perplexity :  30.581636330588506
topic 수 :  12   Coherence Score :  0.40683351915908855   Perplexity :  30.657629838569555
topic 수 :  13   Coherence Score :  0.4382962693299056   Perplexity :  31.951255113305177
topic 수 :  14   Coherence Scor